<a href="https://colab.research.google.com/github/Aldevand/buzzer-detecction-lstm/blob/main/buzzer_detection_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import library

In [16]:
!pip install transformers
!pip install torch
!pip install tensorflow
!pip install keras
import pandas as pd
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from datetime import timedelta


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


# Load Dataset

In [9]:
df = pd.read_excel('/content/drive/MyDrive/Keperluan Skripsi/danantara_gabungan.xlsx')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1255 entries, 0 to 1254
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   conversation_id_str      1255 non-null   int64 
 1   created_at               1255 non-null   object
 2   favorite_count           1255 non-null   int64 
 3   full_text                1255 non-null   object
 4   id_str                   1255 non-null   int64 
 5   image_url                672 non-null    object
 6   in_reply_to_screen_name  112 non-null    object
 7   lang                     1255 non-null   object
 8   location                 810 non-null    object
 9   quote_count              1255 non-null   int64 
 10  reply_count              1255 non-null   int64 
 11  retweet_count            1255 non-null   int64 
 12  tweet_url                1255 non-null   object
 13  user_id_str              1255 non-null   int64 
 14  username                 1255 non-null  

In [11]:
print(df.head(5))
print(df.tail(5))

   conversation_id_str                      created_at  favorite_count  \
0  1900102038295633920  Thu Mar 13 08:30:02 +0000 2025               2   
1  1900122136087585024  Thu Mar 13 09:49:54 +0000 2025               0   
2  1900086851337413120  Thu Mar 13 07:29:41 +0000 2025               0   
3  1898156438125768960  Fri Mar 07 23:38:55 +0000 2025               1   
4  1899762836152155904  Wed Mar 12 10:02:10 +0000 2025               1   

                                           full_text               id_str  \
0  Mengacu dari Indonesia Salary Guide 2025 gaji ...  1900102038295633920   
1  Electrizen Danantara Indonesia telah resmi dib...  1900122136087585024   
2  Wamenkeu Thomas: Danantara Tak Gadai Saham Pem...  1900086851337413120   
3  Proyek DME: Mahal &amp; Merugikan! Gasifikasi ...  1898156438125768960   
4  Masyarakat Indonesia percaya bahwa BPI Dananta...  1899762836152155904   

                                         image_url in_reply_to_screen_name  \
0             

# Text Cleaning

In [14]:
# fungsi pembersihan teks

def cleaning(teks):
  teks = teks.lower()
  teks = re.sub(r'http\S+|www\S+|https\S+', '', teks, flags=re.MULTILINE)
  teks = re.sub(r'@\w+|#\w+', '', teks)
  teks = re.sub(r'[^a-zA-Z\s]', '', teks)
  tokens = word_tokenize(teks)
  list_stopwords = stopwords.words('indonesian')
  filtered_tokens = [token for token in tokens if token not in list_stopwords]
  teks = ' '.join(filtered_tokens)
  return teks
# Terapkan fungsi
df['teks_bersih'] = df['full_text'].apply(cleaning)
print(df[['full_text', 'teks_bersih']].head(5))

                                           full_text  \
0  Mengacu dari Indonesia Salary Guide 2025 gaji ...   
1  Electrizen Danantara Indonesia telah resmi dib...   
2  Wamenkeu Thomas: Danantara Tak Gadai Saham Pem...   
3  Proyek DME: Mahal &amp; Merugikan! Gasifikasi ...   
4  Masyarakat Indonesia percaya bahwa BPI Dananta...   

                                         teks_bersih  
0  mengacu indonesia salary guide gaji selevel ce...  
1  electrizen danantara indonesia resmi dibentuk ...  
2   wamenkeu thomas danantara gadai saham pemerintah  
3  proyek dme mahal amp merugikan gasifikasi batu...  
4  masyarakat indonesia percaya bpi danantara men...  


# Pelabelan -> Rule Based

In [18]:
# Rule waktu
df['created_at'] = pd.to_datetime(df['created_at'], utc=True)
df.sort_values(by='created_at', inplace=True)
df['label_time'] = 0
time_threshold = timedelta(minutes=5)
for i in range(1, len(df)):
    time_diff = df.iloc[i]['created_at'] - df.iloc[i-1]['created_at']
    if time_diff <= time_threshold:
        df.loc[df.index[i], 'label_time'] = 1
        df.loc[df.index[i-1], 'label_time'] = 1